In [15]:
# Imports
import pathlib
import os
from dataclasses import dataclass, field
from typing import Dict, Literal, Optional, Tuple

import imutils
import numpy as np
import cv2
from dataclasses_json import DataClassJsonMixin
import pandas as pd
from tqdm import tqdm

# Constants 
CWD = pathlib.Path(os.path.abspath(""))
GIT_ROOT = CWD.parent
DATA_DIR = GIT_ROOT / "data" / 'PhotosynthesisFall2022'

In [2]:
# Load the data
vid_file = DATA_DIR / 'videos' / 'day 11' / "day11-screen-recording.mp4"
assert vid_file.exists()
cap = cv2.VideoCapture(str(vid_file))
fps = cap.get(cv2.CAP_PROP_FPS)

game_state_file = DATA_DIR / 'game_state' / 'day_11_game_state.csv'
assert game_state_file.exists()
game_state_logs = pd.read_csv(game_state_file)

# Generate timestamp
game_state_logs['datetime'] = pd.to_datetime(game_state_logs['datetime'], format='%Y-%m-%d %H:%M:%S.%f')
game_state_logs['timestamp'] = (game_state_logs['datetime'] - game_state_logs['datetime'].iloc[0]).dt.total_seconds()

In [23]:
# Game state reconstruction routines
@dataclass
class Participant(DataClassJsonMixin):
    id: str
    position: Tuple[float, float]
    state: Literal['null', 'H2O', 'CO2', 'Sugar', 'O2', 'Thinking_H2O'] = 'null'

@dataclass
class EnvironmentState(DataClassJsonMixin):
    sun_state: Optional[bool] = None

@dataclass
class GameState(DataClassJsonMixin):
    participants: Dict[str, Participant] = field(default_factory=dict)
    environment: EnvironmentState = field(default_factory=EnvironmentState)

# Transformation constants
CORRECTIONS = {'OFFSET': (-160,-140), 'AFFINE': (2.2,2)}
START_INDEX = 6000
TIME_OFFSET = 0
N = 5000

def xy_transforms(xy: Tuple[float, float], w: int, h:int) -> Tuple[float, float]:
    x, y = xy
    xx = int(((x+1)*h/2+CORRECTIONS['OFFSET'][0])*CORRECTIONS['AFFINE'][0])
    yy = int(((y+1)*w/2+CORRECTIONS['OFFSET'][1])*CORRECTIONS['AFFINE'][1])
    return (xx, yy)

def render(game_state: GameState, frame: np.ndarray):

    if game_state.participants:
        for p in game_state.participants.values():
            p.position = (eval(p.position[0]), eval(p.position[1]))
            xy = xy_transforms(p.position, frame.shape[0], frame.shape[1])
            frame = cv2.circle(frame, xy, 20, (0, 0, 255), 1)
            frame = cv2.putText(
                frame, 
                p.id, 
                (xy[0]-17, xy[1]), 
                cv2.FONT_HERSHEY_SIMPLEX, 
                0.3, 
                (0,0,255), 
                1, 
                cv2.LINE_AA
            )
            frame = cv2.putText(
                frame, 
                p.state, 
                (xy[0]-8, xy[1]+13), 
                cv2.FONT_HERSHEY_SIMPLEX, 
                0.4, 
                (0,0,255), 
                1, 
                cv2.LINE_AA
            )

    return frame

# Reset video and logs
cap.set(cv2.CAP_PROP_POS_FRAMES, START_INDEX)
game_state_pointer = 0

for i in tqdm(range(N), total=N):
    
    ret, frame = cap.read()
    if not ret:
        break

    frame_timestamp = (START_INDEX + i) / fps
    frame = cv2.putText(
                frame, 
                f"{pd.Timestamp(frame_timestamp, unit='s').strftime('%H:%M:%S.%f')}", 
                (30,30), 
                cv2.FONT_HERSHEY_SIMPLEX, 
                1, 
                (0,0,255), 
                1, 
                cv2.LINE_AA
            )

    while game_state_logs['timestamp'].iloc[game_state_pointer+1] < frame_timestamp + TIME_OFFSET:
        game_state_pointer += 1
    game_state = GameState.from_json(game_state_logs.iloc[game_state_pointer].state)

    frame = render(game_state, frame)
    cv2.imshow('frame', imutils.resize(frame, width=1000))
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()

 14%|█▍        | 707/5000 [00:07<00:43, 99.29it/s] 
